# Coding up a Simple Neural Network from Scratch

In this demo we will code up a simple neural network using (i) PyTorch and (ii) from scratch. Then we will demonstrate that both produce _identical_ outcome.


## A simple DNN from scratch

In [1]:
import numpy as np
import random

class OurLinear:

    def __init__(self):
        self.w = random.uniform(-1, 1)      # randomly initialize learnable weight
        self.b = random.uniform(-1, 1)      # randomly initialize learnable bias
        
    def forward(self, x):
        self.x = x                          # save the input for gradient computation later
        out = x * self.w + self.b           # compute output
        return out
    
    def backward(self, grad_out, lr):
        grad_w = grad_out * self.x          # compute gradients w.r.t. weight
        grad_b = grad_out                   # compute gradients w.r.t. bias
        grad_in = grad_out * self.w         # compute gradients w.r.t. input
        self.w = self.w - lr * grad_w       # update weight
        self.b = self.b - lr * grad_b       # update bias
        return grad_in

In [2]:
class OurTanh:
        
    def forward(self, x):
        self.y = np.tanh(x)                     # compute and save output for gradient computation later
        return self.y
    
    def backward(self, grad_out, lr):
        grad_in = grad_out * (1 - self.y**2)    # compute gradients w.r.t. input
        return grad_in

In [3]:
class OurDNN:

    def __init__(self, num_layers):
        self.layers = []
        for i in range(num_layers):                     # define a sequence of linear and non-linear layers
            self.layers.append(OurLinear())
            self.layers.append(OurTanh())
        
    def forward(self, x):
        for layer in self.layers:                       # run layers sequentially
            x = layer.forward(x)
        return x
    
    def backward(self, grad_out, lr):
        for layer in reversed(self.layers):
            grad_out = layer.backward(grad_out, lr)     # backpropagate gradients through the layers
        return grad_out

## A simple DNN in PyTorch

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class PyDNN(torch.nn.Module):
    
    def __init__(self, num_layers):
        super(PyDNN, self).__init__()
        layers = []
        for i in range(num_layers):             # define a sequence of linear and non-linear layers
            layers.append(nn.Linear(1, 1))
            layers.append(nn.Tanh())
        self.model = nn.Sequential(*layers)     # wrapping it for easier registration and invocation

    def forward(self, x):
        return self.model(x)

## A simple mean squared loss from scratch

In [5]:
class OurMSELoss:

    def forward(self, x, y):
        self.z = (y - x)        # save the intermediate for gradient computation later
        out = self.z**2         # compute output
        return out
    
    def backward(self):
        grad_in = -2 * self.z   # compute gradients w.r.t. input
        return grad_in

## A simple dataset

In [6]:
from torch.utils import data

class SimpleDataset(data.Dataset):

    def __init__(self, func, num_samples=50):
        super(SimpleDataset, self).__init__()
        self.num_samples = num_samples
        xs = [random.uniform(-1, 1) for i in range(self.num_samples)]   # create random values for x
        xys = [([x], [func(x)]) for x in xs]                            # compute y for each x
        xys = [(np.asarray(x, dtype=np.float32),
                np.asarray(y, dtype=np.float32)) for (x, y) in xys]     # convert to numpy array
        self.samples = [(torch.from_numpy(x),            
                         torch.from_numpy(y)) for (x, y) in xys]        # convert to torch tensor

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

### Methods to get and set DNN parameters

In [7]:
def get_py_params(py_dnn):
    params = {}
    params['ws'] = []
    params['bs'] = []
    for name, param in py_dnn.named_parameters():
        if param.requires_grad:
            if name.endswith('weight'):
                params['ws'].append(param.data[0].item())
            elif name.endswith('bias'):
                params['bs'].append(param.data[0].item())
    return params

def get_our_params(our_dnn):
    params = {}
    params['ws'] = []
    params['bs'] = []
    for layer in our_dnn.layers:
        if isinstance(layer, OurLinear):
            params['ws'].append(layer.w)
            params['bs'].append(layer.b)
    return params

def format_params(params):
    return 'params = {}'.format({k : [round(x, 3) for x in v] for k, v in params.items()})

def copy_params(py_dnn, our_dnn):
    py_params = get_py_params(py_dnn)
    for i in range(len(py_params['ws'])):
        our_dnn.layers[i].w = py_params['ws'][i]
        our_dnn.layers[i].b = py_params['bs'][i]

## Define the function we want to learn

In [8]:
def some_function_we_want_to_learn(x):
    return np.tanh(0.6 * x + 0.9)

## Initialize data loader and models

In [9]:
from torch.utils.data import DataLoader

num_layers = 1
lr = 0.01
epoch_size = 50
num_epochs = 10

dataset = SimpleDataset(some_function_we_want_to_learn, num_samples = epoch_size * num_epochs)
dataloader = DataLoader(dataset, shuffle=False, batch_size=1)

py_dnn = PyDNN(num_layers)
our_dnn = OurDNN(num_layers)

## Initialize both models to the same random start state

In [10]:
copy_params(py_dnn, our_dnn)

## Train PyDNN model

In [11]:
py_params = get_py_params(py_dnn)
py_loss = nn.MSELoss()
py_opt = optim.SGD(py_dnn.parameters(), lr=lr)
py_dnn.train()
batch_idx = 0

print('[before training]\t{}'.format(format_params(py_params)))
for _, (x, y) in enumerate(dataloader):
    py_opt.zero_grad()              # in PyTorch the gradients are accumulated, so we zero them before each epoch
    out = py_dnn(x)                 # forward pass over the network
    loss = py_loss(out, y)          # compute the loss
    loss.backward()                 # compute the gradients
    py_opt.step()                   # update model parameters
    batch_idx += 1
    py_params = get_py_params(py_dnn)
    if batch_idx % epoch_size == 0:
        print('[after batch {}] loss = {:.3f} {}'.format(batch_idx, loss.item(), format_params(py_params)))

[before training]	params = {'ws': [-0.493], 'bs': [0.441]}
[after batch 50] loss = 0.025 params = {'ws': [-0.324], 'bs': [0.669]}
[after batch 100] loss = 0.037 params = {'ws': [-0.219], 'bs': [0.759]}
[after batch 150] loss = 0.024 params = {'ws': [-0.142], 'bs': [0.798]}
[after batch 200] loss = 0.004 params = {'ws': [-0.077], 'bs': [0.816]}
[after batch 250] loss = 0.030 params = {'ws': [-0.017], 'bs': [0.83]}
[after batch 300] loss = 0.000 params = {'ws': [0.03], 'bs': [0.84]}
[after batch 350] loss = 0.122 params = {'ws': [0.074], 'bs': [0.847]}
[after batch 400] loss = 0.023 params = {'ws': [0.124], 'bs': [0.828]}
[after batch 450] loss = 0.012 params = {'ws': [0.161], 'bs': [0.827]}
[after batch 500] loss = 0.002 params = {'ws': [0.197], 'bs': [0.809]}


## Train the OurDNN model

In [12]:
our_params = get_our_params(our_dnn)
our_loss = OurMSELoss()
batch_idx = 0

print('[before training]\t{}'.format(format_params(our_params)))
for _, (x, y) in enumerate(dataloader):
    out = our_dnn.forward(x.item())         # forward pass over the network
    loss = our_loss.forward(out, y.item())  # compute the loss
    grad = our_loss.backward()              # compute the gradients with respect to the model output
    our_dnn.backward(grad, lr)              # compute the gradients and update model parameters
    batch_idx += 1
    our_params = get_our_params(our_dnn)
    if batch_idx % epoch_size == 0:
        print('[after batch {}] loss = {:.3f} {}'.format(batch_idx, loss, format_params(our_params)))

[before training]	params = {'ws': [-0.493], 'bs': [0.441]}
[after batch 50] loss = 0.025 params = {'ws': [-0.324], 'bs': [0.669]}
[after batch 100] loss = 0.037 params = {'ws': [-0.219], 'bs': [0.759]}
[after batch 150] loss = 0.024 params = {'ws': [-0.142], 'bs': [0.798]}
[after batch 200] loss = 0.004 params = {'ws': [-0.077], 'bs': [0.816]}
[after batch 250] loss = 0.030 params = {'ws': [-0.017], 'bs': [0.83]}
[after batch 300] loss = 0.000 params = {'ws': [0.03], 'bs': [0.84]}
[after batch 350] loss = 0.122 params = {'ws': [0.074], 'bs': [0.847]}
[after batch 400] loss = 0.023 params = {'ws': [0.124], 'bs': [0.828]}
[after batch 450] loss = 0.012 params = {'ws': [0.161], 'bs': [0.827]}
[after batch 500] loss = 0.002 params = {'ws': [0.197], 'bs': [0.809]}
